In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [5]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

# 데이터 불러오기

In [6]:
os.chdir('/content/drive/Othercomputers/내 MacBook Air/MLDL/project/open')

In [7]:
train = pd.read_csv('./train.csv')
val = pd.read_csv('./val.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [8]:
train_total = train.iloc[:, 1:]
val_total = val.iloc[:, 1:]
test_total = test.iloc[:, 1:]

In [9]:
val_total_x = val_total.drop(['Class'], axis=1)
val_total_y = val_total.loc[:, 'Class']

In [10]:
from sklearn.preprocessing import StandardScaler  # 표준화 
std = StandardScaler()

train_total_std = std.fit_transform(train_total) 
val_total_x_std = std.fit_transform(val_total_x) 
test_total_std = std.fit_transform(test_total) 

In [17]:
from sklearn.decomposition import PCA
def pca_df(train, data):
  pca = PCA(n_components=25) # 주성분 개수
  printcipalComponents = pca.fit(train)
  pca_data = printcipalComponents.transform(data)
  pca_df = pd.DataFrame(pca_data)
  return pca_df

pca_train = pca_df(train_total_std, train_total_std)
pca_val_x = pca_df(train_total_std, val_total_x_std)
pca_test = pca_df(train_total_std, test_total_std)

## EllipticEnvelope

#### 라벨링
* +1 이면 boundary 안에 들어온 값으로 정상 데이터
* -1 이면 outlier

정상데이터: 0 , 이상치: 1로 라벨링 수정

In [18]:
def get_pred_label(model_pred):
  model_pred = np.where(model_pred==1, 0, model_pred)
  model_pred = np.where(model_pred==-1, 1, model_pred)
  return model_pred

### EE Model Fit

In [19]:
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report
EE_model = EllipticEnvelope(support_fraction = 0.994, contamination = 0.00112, random_state = 42)

EE_model.fit(pca_train)

EllipticEnvelope(contamination=0.00112, random_state=42, support_fraction=0.994)

### Predict

In [20]:
val_EE_pred = EE_model.predict(pca_val_x)
val_EE_pred = get_pred_label(val_EE_pred)
val_EE_score = f1_score(val_total_y, val_EE_pred, average='macro')
print(f'Validation F1 Score : [{val_EE_score}]')
print(classification_report(val_total_y, val_EE_pred))

Validation F1 Score : [0.5328409304070532]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.07      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462



## AutoEncoder

In [21]:
pca_train.shape

(113842, 25)

In [22]:
import tensorflow as tf
from tensorflow.keras import Model
# Sequential

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers
# Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU

### Model

In [112]:
input_dim = pca_train.shape[1]
output_dim = input_dim
encoding_dim = 128

input_layer = layers.Input(shape=(input_dim, ))

encoder1 = layers.Dense(encoding_dim/2, name='encoder1')(input_layer)
encoder1 = layers.BatchNormalization()(encoder1)
encoder1 = layers.LeakyReLU()(encoder1)

encoder2 = layers.Dense(encoding_dim, name='encoder2')(encoder1)
encoder2 = layers.BatchNormalization()(encoder2)
encoder2 = layers.LeakyReLU()(encoder2)

decoder1 = layers.Dense(encoding_dim/2, name='decoder1')(encoder2)
decoder1 = layers.BatchNormalization()(decoder1)
decoder1 = layers.LeakyReLU()(decoder1)

decoder2 = layers.Dense(output_dim, name='decoder2')(decoder1)

autoencoder = Model(input_layer, decoder2)
autoencoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

In [113]:
print("====== 모델 전체 구조 ======")
print(autoencoder.summary())

====== 모델 전체 구조 ======
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 25)]              0         
                                                                 
 encoder1 (Dense)            (None, 64)                1664      
                                                                 
 batch_normalization_6 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 64)                0         
                                                                 
 encoder2 (Dense)            (None, 128)               8320      
                                                                 
 batch_normalization_7 (Batc  (None, 128)              512       
 hNormalization)                    

### Fit

In [114]:
EPOCHS = 400
BATCH = 2**14
reduce_lr = ReduceLROnPlateau(monitor='loss', mode='max', factor=0.5, patience=10, min_lr=1e-8, verbose=True)
# es = EarlyStopping(monitor='loss', patience=5, verbose=1)

In [115]:
hist = autoencoder.fit(pca_train, pca_train, 
                 epochs=EPOCHS,
                 batch_size=BATCH,
                 callbacks=[reduce_lr],
                 shuffle=True)

Epoch 1/400
7/7 [==============================] - 3s 17ms/step - loss: 0.7011 - lr: 0.0100
Epoch 2/400
7/7 [==============================] - 0s 13ms/step - loss: 0.1992 - lr: 0.0100
Epoch 3/400
7/7 [==============================] - 0s 14ms/step - loss: 0.1074 - lr: 0.0100
Epoch 4/400
7/7 [==============================] - 0s 14ms/step - loss: 0.0803 - lr: 0.0100
Epoch 5/400
7/7 [==============================] - 0s 14ms/step - loss: 0.0619 - lr: 0.0100
Epoch 6/400
7/7 [==============================] - 0s 14ms/step - loss: 0.0510 - lr: 0.0100
Epoch 7/400
7/7 [==============================] - 0s 13ms/step - loss: 0.0431 - lr: 0.0100
Epoch 8/400
7/7 [==============================] - 0s 12ms/step - loss: 0.0381 - lr: 0.0100
Epoch 9/400
7/7 [==============================] - 0s 15ms/step - loss: 0.0341 - lr: 0.0100
Epoch 10/400
7/7 [==============================] - 0s 12ms/step - loss: 0.0313 - lr: 0.0100
Epoch 11/400
6/7 [========================>.....] - ETA: 0s - loss: 0.0289
Epoc

### Predict 예측

In [116]:
val_y_pred = autoencoder.predict(pca_val_x)

890/890 [==============================] - 2s 2ms/step


#### 예측값 코사인유사도 계산

In [117]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [118]:
def cosine_similarity(true, pred):
  diff = []
  for i in range(len(true)):
    diff.append(cos_sim(true.iloc[i,:], pred[i]))
  diff = pd.Series(diff)
  return np.array(diff)

#### 예측 라벨링

In [119]:
val_AE_pred = np.where(cosine_similarity(pca_val_x, val_y_pred)>0.966, 0, 1)

In [120]:
pd.DataFrame(val_total_y[val_total_y != val_AE_pred]).value_counts()

Class
1        22
0         2
dtype: int64

In [121]:
val_AE_score = f1_score(val_total_y, val_AE_pred, average='macro')
print(f'Validation F1 Score : [{val_AE_score}]')
print(classification_report(val_total_y, val_AE_pred))

Validation F1 Score : [0.6997890443710006]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.80      0.27      0.40        30

    accuracy                           1.00     28462
   macro avg       0.90      0.63      0.70     28462
weighted avg       1.00      1.00      1.00     28462



In [80]:
# (0,0)=>0  | (1,0)=>1  | (0,1)=>0  | (1,1)=>1
from collections import Counter

def mode (x) :
    cnt = Counter(x)
    mode = cnt.most_common(1)
    return mode[0][0]

In [90]:
def get_ensemble_pred(test):
  # AE
  print('AE_model')
  test_AE_pred = autoencoder.predict(test)
  test_AE_pred = np.where(cosine_similarity(test, test_AE_pred)>0.966, 0, 1)

  # pred EE
  print('EE_model')
  test_EE_pred = EE_model.predict(test)
  test_EE_pred = get_pred_label(test_EE_pred)

  preds = pd.DataFrame(zip(test_EE_pred, test_AE_pred))
  preds.columns = ['pred_EE', 'pred_AE']
  
  return preds, preds.apply(mode,axis = 1)

In [91]:
test_pred = get_ensemble_pred(pca_test)[1]

AE_model
4454/4454 [==============================] - 9s 2ms/step
EE_model


In [92]:
test_pred.value_counts()

0    142333
1       170
dtype: int64

In [93]:
val_pred = get_ensemble_pred(pca_val_x)[1]

AE_model
890/890 [==============================] - 2s 2ms/step
EE_model


In [94]:
pd.DataFrame(val_total_y[val_total_y != val_EE_pred]).value_counts()

Class
0        28
1        28
dtype: int64

In [95]:
pd.DataFrame(val_total_y[val_total_y != val_pred]).value_counts()

Class
0        28
1        28
dtype: int64

In [96]:
val_score = f1_score(val_total_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_total_y, val_pred))

Validation F1 Score : [0.5328409304070532]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.07      0.07      0.07        30

    accuracy                           1.00     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       1.00      1.00      1.00     28462

